In [ ]:
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils import get_column_letter
import os

In [79]:
#READ ALL FILE NAMES IN A DIRECTORY
def read_file_names(directory):
    file_names = []
    for file in os.listdir(directory):
        if file.endswith(".xlsx"):
            file_names.append(file)
    return file_names

file_names = read_file_names(r"Filtered_By_Region")

#list all the file names
file_names

['BARMM_NEWCON.xlsx', 'BARMM_REPAIR.xlsx']

In [80]:
# Read all .xlsx files in all subfolders of 'Filtered_By_Region'
all_files = []
for root, dirs, files in os.walk("Filtered_By_Region"):
    for file in files:
        if file.endswith(".xlsx"):
            all_files.append(os.path.join(root, file))

all_files

['Filtered_By_Region\\BARMM_NEWCON.xlsx',
 'Filtered_By_Region\\BARMM_REPAIR.xlsx',
 'Filtered_By_Region\\CAR\\CAR_ELECTRIFICATION.xlsx',
 'Filtered_By_Region\\CAR\\CAR_GABALDON.xlsx',
 'Filtered_By_Region\\CAR\\CAR_HEALTH.xlsx',
 'Filtered_By_Region\\CAR\\CAR_LMS.xlsx',
 'Filtered_By_Region\\CAR\\CAR_NEWCON.xlsx',
 'Filtered_By_Region\\CAR\\CAR_QRF.xlsx',
 'Filtered_By_Region\\CAR\\CAR_REPAIR.xlsx',
 'Filtered_By_Region\\CAR\\CAR_SF.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_ALS-CLC.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_ELECTRIFICATION.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_GABALDON.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_HEALTH.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_LMS.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_NEWCON.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_QRF.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_REPAIR.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_SF.xlsx',
 'Filtered_By_Region\\NCR\\NCR_ELECTRIFICATION.xlsx',
 'Filtered_By_Region\\NCR\\NCR_GABALDON.

In [93]:
for file in all_files:
    if "LMS" in file or "NEWCON" in file:
        print("LMS and NEWCON files are present in the directory.")

LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are present in the directory.
LMS and NEWCON files are presen

In [ ]:
# Load workbook and worksheet
wb = load_workbook(r"Filtered_By_Region\CAR\CAR_NEWCON.xlsx")
ws = wb.active

# Add a new worksheet (or reuse if exists)
if "DropdownOptions" not in wb.sheetnames:
    dropdown_ws = wb.create_sheet("DropdownOptions")
else:
    dropdown_ws = wb["DropdownOptions"]

# Example long list of options
newcon_dropdown_options = [
     r"0% - 10%: Foundation completed: Groundwork finished; no vertical structure yet.",
    r"11% - 25%: Structure and rough-in started: Structural framing in progress; initial MEP rough-in.",
    r"26% - 50%: Structure erected, partial roofing: Building shape defined; roof and systems advancing.",
    r"51% - 75%: Exterior sealed, interior work underway: Enclosed structure; painting, flooring, and testing begin.",
    r"76% - 90%: Final finishes and inspections: Systems tested; ",
    r"91% - 99%: Final touches and punch list: Minor adjustments; final inspections and approvals.",
    r"100% - Construction complete: Ready for handover and occupancy."
]

other_dropdown_options = [
     r"0% - 10%",
    r"11% - 25%",
    r"26% - 50%",
    r"51% - 75%",
    r"76% - 90%",
    r"91% - 99%",
    r"100%"
]

# Write the options into the helper sheet starting at A1
for i, option in enumerate(newcon_dropdown_options, start=1):
    dropdown_ws.cell(row=i, column=1, value=option)

# Hide the helper sheet (optional)
dropdown_ws.sheet_state = 'hidden'

#  Define the range where options are stored
options_range = f"DropdownOptions!$A$1:$A${len(newcon_dropdown_options)}"


In [78]:

#create sample dropdown list in column A
# Create a DataValidation object for the dropdown list

# Add column at the end
new_col_index = ws.max_column + 1
ws.insert_cols(new_col_index)
ws.cell(row=1, column=new_col_index).value = "NEWCON Status"



dv = DataValidation(
    type="list",
    formula1=f"={options_range}", # Reference the options in the helper sheet
    allow_blank=True
)

# Apply validation
ws.add_data_validation(dv)
col_letter = get_column_letter(ws.max_column)
dv.add(f"{col_letter}2:{col_letter}{ws.max_row}")

# sAVE WORKSHEET
wb.save(r"dropdown_example.xlsx")
# 

